# Controle Continu - Martin BAUS

## Apprentissage Automaique I

### Master 2 IAAA

# Utile

Montage de votre Google drive

In [ ]:
import os

from google.colab import drive
drive.mount('drive')

Mounted at drive


## Données

1. Copiez les données dans votre drive.  

2. Déplacez vous dans viotre répertoire avec

```
%cd Repertoire
```

3.  Récupérez les données avec :


```
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

print (X_train.shape, X_test.shape,y_train.shape, y_test.shape)
```


In [5]:
##fait en local (a adapter pour colab)

import numpy as np

X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

In [6]:
X_train 

array([[ 0.97515687,  1.07735128, -0.79017855, ...,  1.43500101,
        -1.13794336, -0.53307194],
       [-1.09497981,  1.2748254 ,  1.25594456, ..., -0.55983705,
         0.31050261,  0.33851792],
       [ 1.00578019, -0.36321   ,  0.14188292, ..., -0.65482934,
        -1.19029683, -0.83149673],
       ...,
       [-2.13617282, -0.00315795,  0.18600417, ..., -1.27821624,
         0.79331793,  0.51852018],
       [-0.32939672,  0.22682429,  1.42691442, ...,  0.29509355,
         0.64789163,  1.32379342],
       [ 0.552555  , -1.25348939, -1.17072436, ...,  0.90660641,
        -0.78310319, -1.34781899]])

---


# 1. Comparez quelques classifieurs
Et  déterminez celui qui vous semble le plus performant en généralisation en utilisant gridsearch, cross valdation et tout ce qui peut vous paraitre utile pour que cette sélection du meilleur modèle soit pertinente.

Produisez tableau de résultats, courbes de performance... et ce que vous trouvez pertinent pour justifier le modèle selectionné.


In [39]:
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.datasets import load_wine
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier




ImportError: HalvingRandomSearchCV is experimental and the API might change without any deprecation cycle. To use it, you need to explicitly import enable_halving_search_cv:
from sklearn.experimental import enable_halving_search_cv

In [41]:

#mise en forme pour comparer classifieurs
classifiers = {
    'KNeighborsClassifier': (KNeighborsClassifier(), {'n_neighbors': [3, 5, 7, 9]}),
    'DecisionTreeClassifier': (DecisionTreeClassifier(), {'max_depth': [3, 5, 7, 9]}),
    'SVC': (SVC(), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}),
    'RandomForestClassifier': (RandomForestClassifier(), {'n_estimators': [10, 50, 100, 200], 'max_depth': [3, 5, 7, 9]}),

    
}

results = []

#Pour chaque classifieur on teste les différents modèles
for clf_name, (clf, param_grid) in classifiers.items():
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train.ravel())
    scores= cross_val_score(grid_search, X_train, y_train.ravel(), cv=5)
    ####logistic = RandomizedSearchCV(estimator='').fit(X_train, y_train.ravel())
    
    

    
    # On stocke les résultats
    results.append({
        'classifier': clf_name,
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'cv_results': grid_search.cv_results_,
        'scores': scores,
        
    })

# Print the results
for result in results:
    print(f"classifieur: {result['classifier']}")
    print(f"Meilleur Paramètre: {result['best_params']}")
    print(f"Best score Grid Search: {result['best_score']}")
    
    print(f"Best score Cross Validation: {max(result['scores'])}")
   
    print("\n")




classifieur: KNeighborsClassifier
Meilleur Paramètre: {'n_neighbors': 9}
Best score Grid Search: 0.8220000000000001
Best score Cross Validation: 0.86


classifieur: DecisionTreeClassifier
Meilleur Paramètre: {'max_depth': 5}
Best score Grid Search: 0.692
Best score Cross Validation: 0.8


classifieur: SVC
Meilleur Paramètre: {'C': 10, 'kernel': 'linear'}
Best score Grid Search: 0.836
Best score Cross Validation: 0.85


classifieur: RandomForestClassifier
Meilleur Paramètre: {'max_depth': 9, 'n_estimators': 200}
Best score Grid Search: 0.8219999999999998
Best score Cross Validation: 0.88




Le cross validation semble meilleur que le grid search 

# 2. Elimination de variables inutiles

On vous apprend que 3 variables dans les données (= 3 colonnes de $X$) sont en réalité du bruit et ne contiennent aucune information sur la cible $y$.   

* Question 1)

Pourquoi ces variables de bruit sont-elles un problème ?

* Question 2)

Pour prendre en compte cette information, on vous propose de tester la procdure suivante.

2. Itérer 3 fois
* Apprendre le classifier avec l'ensemble des variables de $X$ et notez le loss obtenu en généralisation (sur une base de validation)
* Pour chacune variable $i$ des données $X$
  *   Apprenez un classifieur et testez sa performance en généralisation en utilisant toutes les variables sauf la $i^{ème}$ variable
  *   Determinez quelle variabe $i^*$ induit le moins de perte dans le loss en généralisation, c'est sans doute une variable de bruit. Vous allez la supprimer.
  * Redéfinissez les données $X$ (en train et en test) en éliminant la variable $i^*$ (vous éliminez la colonne correspondante de la matrice $X_{train}$ et la matrice $X_{test}$)

Implémentez cette stratégie, reportez les résultats obtenus de façon synthétque (tableau, grahique) et identifiez les 3 variables de bruit (donnez les indices des variables trouvées)


Q1 Les variables de bruit sont problématique puisqu'elles prennent des ressources pour être calculées par le modèle (ce qui n'est pas nécéssaire). De plus, elles peuvent fausser le modèle 

Q2

In [43]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss



X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")


bruit = []

for iteration in range(3):
    clf = RandomForestClassifier(n_estimators=100, max_depth=5)
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    base_loss = log_loss(y_test, y_pred)
    
    losses = []
    
    #On calcule la loss en excluant une variable à chaque fois
    for i in range(X_train.shape[1]):
        X_train_reduced = np.delete(X_train, i, axis=1)
        X_test_reduced = np.delete(X_test, i, axis=1)
        
        clf.fit(X_train_reduced, y_train)
        y_pred = clf.predict_proba(X_test_reduced)
        loss = log_loss(y_test, y_pred)
        losses.append((i, loss))
    
    # On identifie la variable avec la plus petite loss pour trouver le bruit 
    i_star, min_loss = min(losses, key=lambda x: x[1])
    bruit.append(i_star)
    
    #avant de commencer une nouvelle itération on enlève la colonne bruitée pour trouver le 2ème bruit
    X_train = np.delete(X_train, i_star, axis=1)
    X_test = np.delete(X_test, i_star, axis=1)


print("variables de bruit:", bruit)

########
#variables de bruit: [9, 3, 1] pour RandomForest
#variables de bruit: [15, 3, 13] pour KNeighbors
#variables de bruit: [8, 3, 7] pour RandomForestClassifier(n_estimators=100, max_depth=5)
########




c:\Users\bausm\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\bausm\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\bausm\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\bausm\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConvers

variables de bruit: [1, 11, 13]


# Analyse des résultats

L'indentification des variables de bruits semble être fortement dépendante du modèle et classifieur utilisé et de ses paramètres (nous avons testé plusieurs classifieurs et aucun n'au donné les mêmes résultats, cependant la variable n3 était tout le temps présente)